In [1]:
import deepchem as dc

import pandas as pd
import numpy as np

from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

import ast
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.manifold import TSNE
import seaborn as sns
plt.rcParams['figure.dpi'] = 300

from sklearn.preprocessing import LabelEncoder

Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading some PyTorch models, missing a dependency. No module named 'torch'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch'
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'torch'
Skipped loading some Jax models, missing a dependency. No module named 'jax'


In [2]:
data = pd.read_csv('../../data/BST.csv',encoding = 'gb2312')

In [3]:
data

Name  \
0                       Conessine hydrobromide   
1                       Arecoline hydrobromide   
2                                    Avocadyne   
3      1-Acetoxy-2,4-dihydroxyheptadeca-16-yne   
4                             Hydrogen cyanide   
...                                        ...   
13021                               CNP0005404   
13022                               CNP0005405   
13023                               CNP0005406   
13024                               CNP0005407   
13025                               CNP0005408   

                                                  SMILES Flavor type  \
0      Br.Br.CC1C2CCC3C4CC=C5CC(N(C)C)CCC5(C)C4CCC32CN1C      Bitter   
1                                 Br.COC(=O)C1=CCCN(C)C1      Bitter   
2                              C#CCCCCCCCCCCCC(O)CC(O)CO      Bitter   
3                        C#CCCCCCCCCCCCC(O)CC(O)COC(C)=O      Bitter   
4                                                    C#N      Bitter   
...                                                  ...         ...   
13021        O=C1N(CC2=NC=CC=C2)CC3N1CCN(CC=4C=CC=CC4)C3   Tasteless   
13022            O=C1N(CC=2C=NC=CC2)CC3N1CCN(C3)C4CCOCC4   Tasteless   
13023     O=C1N(C=2C=CC=CC2)CC3N1CCN(C3)C4CCN(C(=O)C)CC4   Tasteless   
13024         O=C1N(C=2C=CC=CC2)CC3N1CCN(CC4=NC=CC=C4)C3   Tasteless   
13025  O=C(O)C1=CC=C(C=C1)CN2CCN3C(=O)N(C=4C=CC=CC4)C...   Tasteless   

                                           Data resource Label  
0      BitterSweet, PlantMolecularTasteDB, BitterDB, ...   [1]  
1                     BitterSweet, BitterDB, ChemTasteDB   [1]  
2           PlantMolecularTasteDB, BitterDB, ChemTasteDB   [1]  
3                                  BitterDB, ChemTasteDB   [1]  
4                     BitterSweet, BitterDB, ChemTasteDB   [1]  
...                                                  ...   ...  
13021                                            Coconut   [3]  
13022                                            Coconut   [3]  
13023                                            Coconut   [3]  
13024                                            Coconut   [3]  
13025                                            Coconut   [3]  

[13026 rows x 5 columns]

# 特征提取

In [4]:
featurizer = dc.feat.CircularFingerprint(size=1024,radius=4)
features = featurizer.featurize(data['SMILES'])

#dataset = dc.data.NumpyDataset(features,data['Label'])

In [5]:
from sklearn.feature_selection import VarianceThreshold

# 初始化VarianceThreshold对象
vt = VarianceThreshold(threshold = (.98 * (1 - .98)))

# 进行特征选择
X_new = vt.fit_transform(features)

dataset = dc.data.NumpyDataset(X = X_new,y = data['Label'],ids=data['SMILES'])

## 数据划分

In [6]:
splitter = dc.splits.ScaffoldSplitter()
train_dataset,test_dataset = splitter.train_test_split(dataset=dataset,frac_train=0.8,seed=200)

In [7]:
test_dataset.X

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

# Multi-label

In [8]:
train_dataset_y = [ast.literal_eval(label) for label in train_dataset.y]
test_dataset_y  = [ast.literal_eval(label) for label in test_dataset.y]
train_new_y = MultiLabelBinarizer().fit_transform(train_dataset_y)
test_new_y = MultiLabelBinarizer().fit_transform(test_dataset_y)

In [9]:
def row_counter(my_array):
    list_of_tups = [tuple(ele) for ele in my_array]
    return Counter(list_of_tups)

In [10]:
print('train_dataset:',row_counter(train_new_y))
train_counter = row_counter(train_new_y)
print('Bitter : Sweet : Tasteless = ',train_counter[(1, 0, 0)]/10420,':',train_counter[(0, 1, 0)]/10420,':',train_counter[(0, 0, 1)]/10420)
print('test_dataset:',row_counter(test_new_y))
test_counter = row_counter(test_new_y)
print('Bitter : Sweet : Tasteless = ',test_counter[(1, 0, 0)]/2606,':',test_counter[(0, 1, 0)]/2606,':',test_counter[(0, 0, 1)]/2606)

train_dataset: Counter({(0, 1, 0): 5684, (0, 0, 1): 3510, (1, 0, 0): 1226})
Bitter : Sweet : Tasteless =  0.11765834932821496 : 0.545489443378119 : 0.33685220729366605
test_dataset: Counter({(0, 0, 1): 1471, (1, 0, 0): 720, (0, 1, 0): 415})
Bitter : Sweet : Tasteless =  0.2762854950115119 : 0.15924788948580199 : 0.5644666155026861


# GridSearchCV

In [11]:
scoring = {
     'F1_score': make_scorer(f1_score,average ='macro'),
     'Accuracy': make_scorer(accuracy_score),
     'Recall': make_scorer(recall_score,average = 'macro'),
     'Precision':make_scorer(precision_score,average = 'macro'),
}

param_grid = {'estimator__C': [0.001, 0.01, 0.05, 0.25, 0.5,  2.5, 5, 10], 
               'estimator__kernel': ['rbf'],
              #'estimator__gamma':[0.001,  0.01, 0.05,  0.25, 0.5,  0.7, 0.9, 1],
              }


# param_grid = {'estimator__C': [0.001, 0.005, 0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 1, 2.5, 5, 7.5, 10, 25, 50, 75, 100], 
#               'estimator__kernel': ['rbf'],
#              'estimator__gamma':[0.001, 0.005, 0.01, 0.05, 0.1, 0.25, 0.5, 0.6, 0.7, 0.8, 0.8, 0.9, 1]
#              }

In [12]:
classifier = SVC(probability=True)
ovr = OneVsRestClassifier(classifier)

gs = GridSearchCV(
                    ovr,
                    param_grid,
                    scoring = scoring,
                    cv = KFold(n_splits=5, shuffle=True, random_state=200),
                    n_jobs = -1,
                    refit = 'F1_score',
                    return_train_score = True)

gs_fit = gs.fit(train_dataset.X,train_new_y)

In [13]:
val_F1 = gs.best_score_
val_ACC = gs.cv_results_['mean_test_Accuracy'][gs.best_index_]
val_Recall = gs.cv_results_['mean_test_Recall'][gs.best_index_]
val_Precision = gs.cv_results_['mean_test_Precision'][gs.best_index_]

In [14]:
print('Best parameters: ', gs.best_params_)
print('Best score (F1): ', val_F1)
print('Accuracy:',val_ACC)
print('Recall:',val_Recall)
print('Precision:',val_Precision)

Best parameters:  {'estimator__C': 5, 'estimator__kernel': 'rbf'}
Best score (F1):  0.8458163868281774
Accuracy: 0.8588291746641076
Recall: 0.8122393428205203
Precision: 0.8870913926313406


In [15]:
model_multi = gs_fit.best_estimator_
model_multi

OneVsRestClassifier(estimator=SVC(C=5, probability=True))

# Evaluate model

In [16]:
y_test_pred = model_multi.predict(test_dataset.X)
y_test_pred

array([[0, 1, 0],
       [0, 1, 0],
       [1, 0, 0],
       ...,
       [0, 0, 1],
       [1, 0, 0],
       [0, 0, 0]])

In [17]:
# 将真实标签和预测标签转换为整数形式
y_true = np.argmax(test_new_y, axis=1)
y_pred = np.argmax(y_test_pred, axis=1)

In [18]:
# 将标签转换为整数形式
label_encoder = LabelEncoder()
y_true = label_encoder.fit_transform(y_true)
y_pred = label_encoder.transform(y_pred)

In [19]:
cm = confusion_matrix(y_true, y_pred)

In [20]:
cm

array([[ 499,   32,  189],
       [  82,  298,   35],
       [ 288,   43, 1140]], dtype=int64)

In [21]:
class_names = label_encoder.classes_  # 类别名称

# 绘制混淆矩阵图
class_names = ['Bitter', 'Sweet', 'Tasteless']  # 类别名称

# 标准化混淆矩阵
#cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

# 绘制热力图
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

In [22]:
test_F1 = f1_score(test_new_y, y_test_pred,average ='macro')
test_ACC = accuracy_score(test_new_y, y_test_pred)
test_Recall = recall_score(test_new_y,y_test_pred,average = 'macro')
test_Precision = precision_score(test_new_y, y_test_pred,average = 'macro')

In [23]:
rf_performance_dataset = {
    'F1_score':[val_F1,test_F1],
    'ACC':[val_ACC,test_ACC],
    'Recall':[val_Recall,test_Recall],
    'Precision':[val_Precision,test_Precision],
}

In [24]:
rf_performance = pd.DataFrame(rf_performance_dataset,index=['val','test'])
rf_performance

F1_score       ACC    Recall  Precision
val   0.845816  0.858829  0.812239   0.887091
test  0.689492  0.647352  0.641323   0.760428